Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [1]:
let name = "Snehadeep Gayen"
let rollno = "CS21B078"

Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads



val name : string = "Snehadeep Gayen"


val rollno : string = "CS21B078"


## Important notes about grading:

1. All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please contact the TAs or the instructor or visit the course contact hour. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. All assignments handed in after the deadline will be considered late, and will consume your grace days. 
3. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

## A Prolog interpreter in OCaml

In this assignment, you will implement a Prolog interpreter in OCaml with support for backtracking (recover from bad choices) and choice points (produce multiple results). It is highly recommended to use the functions `List.map` and `List.fold_left`; these are applicable in almost all the problems, and they will significantly simplify the code. In general, you can use any function from any library for this assignment. 

We will start with the definition of terms, predicates, clauses, goals and programs.

In [2]:
type constant = Constant of string
type variable = Variable of string
type func = Function of string * term list
and term = C of constant | V of variable | F of func

type predicate = Predicate of string * term list

type head = predicate
type body = predicate list

type clause = Fact of head | Rule of head * body

type goal = predicate list

type program = clause list

type constant = Constant of string


type variable = Variable of string


type func = Function of string * term list
and term = C of constant | V of variable | F of func


type predicate = Predicate of string * term list


type head = predicate


type body = predicate list


type clause = Fact of head | Rule of head * body


type goal = predicate list


type program = clause list


The following stringification functions will help you in debugging.

In [3]:
let rec string_of_f_list f tl =
  let _, s = List.fold_left (fun (first, s) t -> 
    let prefix = if first then "" else s ^ ", " in
    false, prefix ^ (f t)) (true,"") tl
  in 
  s

let rec string_of_term t =
  match t with
  | C (Constant c) -> c
  | V (Variable v) -> v
  | F (Function (f,tl)) -> 
      f ^ "(" ^ (string_of_f_list string_of_term tl) ^ ")"

let string_of_predicate (Predicate (p,tl)) =
  p ^ "(" ^ (string_of_f_list string_of_term tl) ^")"
  
let string_of_predicate_list pl =
  string_of_f_list string_of_predicate pl
  
let string_of_goal g =
  "?- " ^ (string_of_predicate_list g)

let string_of_clause c =
  match c with
  | Fact f -> string_of_predicate f ^ "."
  | Rule (h,b) -> string_of_predicate h ^ " :- " ^ (string_of_predicate_list b) ^ "."

let string_of_program p =
  List.fold_left (fun acc c -> acc ^ (string_of_clause c) ^ "\n") "" p

val string_of_f_list : ('a -> string) -> 'a list -> string = <fun>


val string_of_term : term -> string = <fun>


val string_of_predicate : predicate -> string = <fun>


val string_of_predicate_list : predicate list -> string = <fun>


val string_of_goal : predicate list -> string = <fun>


val string_of_clause : clause -> string = <fun>


val string_of_program : clause list -> string = <fun>


Below are more helper functions:

In [4]:
let var v = Variable v
let var_t v = V (var v)
let const c = Constant c
let const_t c = C (Constant c)
let func f l = Function (f,l)
let func_t f l = F (func f l)
let pred p l = Predicate (p,l)
let fact f = Fact f
let rule h b = Rule (h,b)

val var : string -> variable = <fun>


val var_t : string -> term = <fun>


val const : string -> constant = <fun>


val const_t : string -> term = <fun>


val func : string -> term list -> func = <fun>


val func_t : string -> term list -> term = <fun>


val pred : string -> term list -> predicate = <fun>


val fact : head -> clause = <fun>


val rule : head -> body -> clause = <fun>


## Problem 1

In this problem, you will implement the function:

```ocaml
occurs_check : variable -> term -> bool
```

`occurs_check v t` returns `true` if the variable `v` occurs in `t`. 

In [5]:
let rec occurs_check v t =
    match v with Variable x ->
    match t with 
    | V (Variable y) ->  x = y
    | F (Function (f, term_list)) -> List.fold_left (fun acc newterm -> (occurs_check v newterm) || acc) false term_list
    | _ -> false
    ;

val occurs_check : variable -> term -> bool = <fun>


In [6]:
(* 5 points *)
assert (occurs_check (var "X") (var_t "X"));
assert (not (occurs_check (var "X") (var_t "Y")));
assert (occurs_check (var "X") (func_t "f" [var_t "X"]))

- : unit = ()


## Problem 2

Implement the following functions which return the variables contained in them:

```ocaml
variables_of_term      : term -> VarSet.t
variables_of_predicate : predicate -> VarSet.t
variables_of_clause    : clause -> VarSet.t
```

For this problem, we will use the `Set` library to maintain sets of variables. The documentation is available here: https://v2.ocaml.org/api/Set.S.html. You will need to use a few functions from this library.

In [7]:
module VarSet = Set.Make(struct type t = variable let compare = Pervasives.compare end)
(* DO NOT CHANGE THE ABOVE DEFINITION 
   You can ignore any alerts that you may get regarding the usage of Pervasives.*)

let rec variables_of_term t = 
  match t with
  | V x -> VarSet.singleton x
  | F (Function (f, term_list)) -> 
          List.fold_left 
              (fun acc newTerm -> VarSet.union acc (variables_of_term newTerm)) 
              VarSet.empty term_list
  | _ -> VarSet.empty
  
  
let rec variables_of_predicate (Predicate (_, l)) =
    List.fold_left 
        (fun acc newTerm -> VarSet.union acc (variables_of_term newTerm))
        VarSet.empty l
        
let variables_of_clause c =
    match c with
    | Fact c_fact_pred -> variables_of_predicate c_fact_pred
    | Rule (c_head, c_body_pred_list) ->
        let initSet = variables_of_predicate c_head in
            List.fold_left (
                fun acc newPred -> VarSet.union acc (variables_of_predicate newPred)
                ) initSet c_body_pred_list
    ;

File "[7]", line 1, characters 64-82:
1 | module VarSet = Set.Make(struct type t = variable let compare = Pervasives.compare end)
                                                                    ^^^^^^^^^^^^^^^^^^
Alert deprecated: module Stdlib.Pervasives
Use Stdlib instead.

If you need to stay compatible with OCaml < 4.07, you can use the 
stdlib-shims library: https://github.com/ocaml/stdlib-shims


module VarSet :
  sig
    type elt = variable
    type t
    val empty : t
    val is_empty : t -> bool
    val mem : elt -> t -> bool
    val add : elt -> t -> t
    val singleton : elt -> t
    val remove : elt -> t -> t
    val union : t -> t -> t
    val inter : t -> t -> t
    val disjoint : t -> t -> bool
    val diff : t -> t -> t
    val compare : t -> t -> int
    val equal : t -> t -> bool
    val subset : t -> t -> bool
    val iter : (elt -> unit) -> t -> unit
    val map : (elt -> elt) -> t -> t
    val fold : (elt -> 'a -> 'a) -> t -> 'a -> 'a
    val for_all : (elt -> bool) -> t -> bool
    val exists : (elt -> bool) -> t -> bool
    val filter : (elt -> bool) -> t -> t
    val partition : (elt -> bool) -> t -> t * t
    val cardinal : t -> int
    val elements : t -> elt list
    val min_elt : t -> elt
    val min_elt_opt : t -> elt option
    val max_elt : t -> elt
    val max_elt_opt : t -> elt option
    val choose : t -> elt
    val choose_opt : t -> elt option
    

val variables_of_term : term -> VarSet.t = <fun>


val variables_of_predicate : predicate -> VarSet.t = <fun>


val variables_of_clause : clause -> VarSet.t = <fun>


In [8]:
(* 5 points *)
assert (variables_of_term (func_t "f" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);
assert (variables_of_predicate (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);
assert (variables_of_clause (fact @@ pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);
assert (variables_of_clause (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]])= 
        VarSet.of_list [var "X"; var "Y"])

- : unit = ()


## Problem 3

The value of type `term Substitution.t` is a OCaml map whose key is of type `variable` and value is of type `term`. In other words, it is map from variables to terms. Implement substitution functions that take a `term Substitution.t` and use that to perform the substitution:

```ocaml
substitute_in_term : term Substitution.t -> term -> term
substitute_in_predicate : term Substitution.t -> predicate -> predicate
substitute_in_clause : term Substitution.t -> clause -> clause
```

You will need to use some functions from the Map library: https://v2.ocaml.org/api/Map.S.html.

In [9]:
module Substitution = Map.Make(struct type t = variable let compare = Pervasives.compare end)

let string_of_substitution s =
  "{" ^ (Substitution.fold (fun (Variable v) t s -> s ^ "; " ^ v ^ " -> " ^ (string_of_term t)) s "") ^ "}"

let rec substitute_in_term s t = 
    match t with
    | C _ -> t
    | V v -> if Substitution.mem v s then Substitution.find v s else t
    | F (Function (f, term_list)) -> (
            let new_term_list = (
                List.fold_right 
                (fun old_term acc -> (substitute_in_term s old_term)::acc)
                term_list
                []
            ) in
            F (Function (f, new_term_list))
        )


let rec substitute_in_predicate s (Predicate (p,l)) = 
    Predicate (p, 
                List.fold_right
                    (fun old_term acc -> (substitute_in_term s old_term)::acc)
                    l
                    []
            )

let substitute_in_clause s c = 
    match c with
    | Fact hd -> Fact (substitute_in_predicate s hd)
    | Rule (hd, body) -> Rule(
                            substitute_in_predicate s hd,
                            List.fold_right
                              (fun old_pred acc -> (substitute_in_predicate s old_pred)::acc)
                              body
                              []
                        )


File "[9]", line 1, characters 70-88:
1 | module Substitution = Map.Make(struct type t = variable let compare = Pervasives.compare end)
                                                                          ^^^^^^^^^^^^^^^^^^
Alert deprecated: module Stdlib.Pervasives
Use Stdlib instead.

If you need to stay compatible with OCaml < 4.07, you can use the 
stdlib-shims library: https://github.com/ocaml/stdlib-shims


module Substitution :
  sig
    type key = variable
    type +'a t
    val empty : 'a t
    val is_empty : 'a t -> bool
    val mem : key -> 'a t -> bool
    val add : key -> 'a -> 'a t -> 'a t
    val update : key -> ('a option -> 'a option) -> 'a t -> 'a t
    val singleton : key -> 'a -> 'a t
    val remove : key -> 'a t -> 'a t
    val merge :
      (key -> 'a option -> 'b option -> 'c option) -> 'a t -> 'b t -> 'c t
    val union : (key -> 'a -> 'a -> 'a option) -> 'a t -> 'a t -> 'a t
    val compare : ('a -> 'a -> int) -> 'a t -> 'a t -> int
    val equal : ('a -> 'a -> bool) -> 'a t -> 'a t -> bool
    val iter : (key -> 'a -> unit) -> 'a t -> unit
    val fold : (key -> 'a -> 'b -> 'b) -> 'a t -> 'b -> 'b
    val for_all : (key -> 'a -> bool) -> 'a t -> bool
    val exists : (key -> 'a -> bool) -> 'a t -> bool
    val filter : (key -> 'a -> bool) -> 'a t -> 'a t
    val partition : (key -> 'a -> bool) -> 'a t -> 'a t * 'a t
    val cardinal : 'a t -> int
    val bindings : 'a 

val string_of_substitution : term Substitution.t -> string = <fun>


val substitute_in_term : term Substitution.t -> term -> term = <fun>


val substitute_in_predicate : term Substitution.t -> predicate -> predicate =
  <fun>


val substitute_in_clause : term Substitution.t -> clause -> clause = <fun>


In [10]:
(* 10 points *)
let s = 
  let open Substitution in 
  add (var "Y") (const_t "0") (add (var "X") (var_t "Y") empty)
in
assert (substitute_in_term s (func_t "f" [var_t "X"; var_t "Y"; const_t "a"]) = 
        func_t "f" [var_t "Y"; const_t "0"; const_t "a"]);
assert (substitute_in_predicate s (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        pred "p" [var_t "Y"; const_t "0"; const_t "a"]);
assert (substitute_in_clause s (fact @@ pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        (fact @@ pred "p" [var_t "Y"; const_t "0"; const_t "a"]));
assert (substitute_in_clause s (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]]) = 
        (rule (pred "p" [var_t "Y"; const_t "0"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]]))

- : unit = ()


We define a function `freshen` that given a clause, returns a clause where are the variables have been renamed with fresh variables. 

In [11]:
let counter = ref 0
let fresh () = 
  let c = !counter in
  counter := !counter + 1;
  V (Variable ("_G" ^ string_of_int c))

let freshen c =
  let vars = variables_of_clause c in
  let s = VarSet.fold (fun v s -> Substitution.add v (fresh()) s) vars Substitution.empty in
  substitute_in_clause s c

val counter : int ref = {contents = 0}


val fresh : unit -> term = <fun>


val freshen : clause -> clause = <fun>


For example, 

In [12]:
let c = (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [var_t "X"; const_t "b"; const_t "a"]])
let _ = string_of_clause c
let _ = string_of_clause (freshen c)

val c : clause =
  Rule
   (Predicate ("p", [V (Variable "X"); V (Variable "Y"); C (Constant "a")]),
   [Predicate ("q", [V (Variable "X"); C (Constant "b"); C (Constant "a")])])


- : string = "p(X, Y, a) :- q(X, b, a)."


- : string = "p(_G0, _G1, a) :- q(_G0, b, a)."


## Problem 4

Implement the function:

```ocaml
compose: term Substitution.t -> term Substitution.t -> term Substitution.t
```

that composes two substitutions. Use the precise definition of composition from the lectures.


In [13]:
let compose s1 s2 = 
    let modified_s1 = 
        Substitution.fold 
        (fun x old_term subs -> 
            let new_term = substitute_in_term s2 old_term in
            if (V x) = new_term then subs else
            (Substitution.add x new_term subs))
        s1
        Substitution.empty 
    in 
    Substitution.fold
    (fun x t subs -> if Substitution.mem x subs then subs else Substitution.add x t subs)
    s2
    modified_s1

val compose :
  term Substitution.t -> term Substitution.t -> term Substitution.t = <fun>


In [14]:
(* 10 points *)
let s1 = 
  let open Substitution in 
  add (var "Z") (const_t "a") (add (var "X") (var_t "Y") empty)
in
let s2 = 
  let open Substitution in 
  add (var "Z") (const_t "b") (add (var "Y") (var_t "X") empty)
in
assert (string_of_substitution (compose s1 s2) = "{; Y -> X; Z -> a}" ||
        string_of_substitution (compose s1 s2) = "{; Z -> a; Y -> X}");
assert (string_of_substitution (compose s2 s1) = "{; X -> Y; Z -> b}" ||
        string_of_substitution (compose s2 s1) = "{; Z -> b; X -> Y}")
        

- : unit = ()


## Problem 4

Implement the function:

```ocaml
mgu : term -> term -> term Substitution.t option
```

which returns the most general unifier of the given terms. The function should return `None` if the terms are not unifiable. You need to precisely implement the algorithm for computing mgu as defined in the lectures.

Using `mgu`, implement the function:

```ocaml
mgu_predicate : predicate -> predicate -> term Substitution.t option
```

which returns the most general unifier of the given predicates. 

In [15]:
let rec mgu_rec old_t1 old_t2 opt_theta = 
    match opt_theta with 
    | None -> None 
    | Some theta -> 
    let t1 = substitute_in_term theta old_t1 in
    let t2 = substitute_in_term theta old_t2 in
    match (t1, t2) with
    | (V v1, V v2) -> 
                    if v1 = v2 
                    then Some (compose theta Substitution.empty)
                    else Some (compose theta (Substitution.singleton v1 t2))
    | (C c1, C c2) -> 
                    if c1 = c2 then Some (compose theta Substitution.empty)
                    else None
    | (V v1, _)    -> 
                    if occurs_check v1 t2 then None
                    else Some (compose theta (Substitution.singleton v1 t2))

    | (_, V v2)    -> 
                    if occurs_check v2 t1 then None
                    else Some (compose theta (Substitution.singleton v2 t1))

    | (F (Function (f1, term_list1)), F (Function (f2, term_list2))) ->

        if f1 = f2 && (List.length term_list1) = (List.length term_list2) then (

        List.fold_left (fun acc (x_i, y_i) -> 
                        let u = mgu_rec x_i y_i acc in (
                            match u with 
                            | None -> (* Printf.printf "None\n%!";*) None
                            | Some uu -> (* Printf.printf "%s\n%!" (string_of_substitution uu);*) u
                        ) ) opt_theta (List.combine term_list1 term_list2)
        )

        else None

    | _ -> None
;;
    
let mgu input_t1 input_t2 =
    mgu_rec input_t1 input_t2 (Some Substitution.empty);;

let mgu_predicate (Predicate (p1,l1)) (Predicate (p2,l2)) =

    if (p1 = p2) && ((List.length l1) = (List.length l2)) then (

            List.fold_left (fun acc (t1, t2) -> 
                                let u = mgu_rec t1 t2 acc in 
                                    match u with 
                                    | None -> (* Printf.printf "None\n%!"; *) None
                                    | Some uu -> (* Printf.printf "%s\n%!" (string_of_substitution uu);*) u
                            )
                         (Some Substitution.empty) (List.combine l1 l2)
        )
    else 
        (
(*         Printf.printf "some big mistake %b %b \n%!" (p1 = p2) ((List.length l1) = (List.length l2));  *)
        None)

val mgu_rec :
  term -> term -> term Substitution.t option -> term Substitution.t option =
  <fun>


val mgu : term -> term -> term Substitution.t option = <fun>


val mgu_predicate : predicate -> predicate -> term Substitution.t option =
  <fun>


In [16]:
(* 15 points *)
assert (string_of_substitution (Option.get(mgu (var_t "X") (var_t "Y"))) = "{; X -> Y}" ||
        string_of_substitution (Option.get(mgu (var_t "X") (var_t "Y"))) = "{; Y -> X}");
assert (string_of_substitution (Option.get(mgu (var_t "Y") (var_t "X"))) = "{; X -> Y}" ||
        string_of_substitution (Option.get(mgu (var_t "Y") (var_t "X"))) = "{; Y -> X}");
assert (Option.get(mgu (var_t "Y") (var_t "Y")) = Substitution.empty);
assert (Option.get(mgu (const_t "0") (const_t "0")) = Substitution.empty);
assert (Option.get(mgu (const_t "0") (var_t "Y")) = Substitution.singleton (var "Y") (const_t "0"));
assert (
  match mgu (const_t "0") (const_t "1") with
  | None -> true
  | _ -> false );
assert (
  match mgu (func_t "f" [const_t "0"]) (func_t "g" [const_t "1"]) with
  | None -> true
  | _ -> false );
assert (Option.get(mgu (func_t "f" [var_t "X"]) (func_t "f" [var_t "Y"])) = Substitution.singleton (var "X") (var_t "Y") ||
        Option.get(mgu (func_t "f" [var_t "X"]) (func_t "f" [var_t "Y"])) = Substitution.singleton (var "Y") (var_t "X"))

- : unit = ()


In [17]:
(* 15 points *)
let t1 = F (Function("f", [V (Variable "X"); V (Variable "Y"); V (Variable "Y")])) in 
let t2 = F (Function("f", [V (Variable "Y"); V (Variable "Z"); V (Variable "a")])) in
let u = Option.get(mgu t1 t2) in
assert (Substitution.cardinal u = 3);
assert (string_of_substitution u = "{; X -> a; Y -> a; Z -> a}");
let t1 = var_t "X" in
let t2 = F (Function("f", [const_t "a"; const_t "b"; var_t "X"])) in
assert (
  match mgu t1 t2 with
  | None -> true
  |_ -> false
)


- : unit = ()


## Problem 5

Implement the function:

```ocaml
query : clause list -> predicate list -> predicate list list 
```

where 

* the first argument is the `program` which is a list of clauses.
* the second argument is the `goal` which is a list of predicates.

The function returns a list of predicate lists (`results`). Each of these results is an instance of the original goal and is a logical consequence of the program. If there exists no instance of given goal which is a logical consequence of the program, then the result is an empty list. See test cases for expected results.

For the rule and goal order, choose what Prolog does; choose the left-most subgoal for goal order and first rule in the order in which the rules appear in the program for the rule order. 

**Hint**: Implement a purely functional recursive solution based on the algorithm from the lectures. The backtracking and choice points naturally fall out of the implementation. In particular, given a goal from the resolvant, you need to find all matching clauses of the program that can be unified with the goal, and recurse on the updated resolvant for each such clause. You will also need to use the `freshen` function on clauses in the program to prevent spurious `occurs_check` during unification. 

In [18]:
let pred_list_subs subs pred_list = 
    List.fold_right (fun pred acc -> (substitute_in_predicate subs pred)::acc) pred_list []

(* Takes a program, goal_list and finds all substitutions for the variables satisfying 
    the program and the goal_list. It returns all the answers possible by applying 
    these substitutions on the original_goal *)
let rec query_rec program goal_list original_goal =
    match goal_list with
    
    | [] -> [ original_goal ]
    
    | goal :: other_goal_list ->
    
(*             Printf.printf "Goal is %s\n%!" (string_of_predicate goal); *)
            
            (* Iterate through clauses *)
            List.fold_left (
                
                fun ans_list a_clause ->
                
(*                     Printf.printf "On clause %s\n" (string_of_clause a_clause); *)
                        
                        match freshen a_clause with 
                        
                        | Fact fct -> (
                        
                            match mgu_predicate fct goal with 
                                        
                                | None -> ans_list (* This clause can't be matched *)

                                | Some subs -> ans_list @ 
                                
                                        (query_rec program
                                            (pred_list_subs subs other_goal_list) 
                                            (pred_list_subs subs original_goal)
                                        )
                            )
                        
                        | Rule (hd, bdy) -> 
                        
                            match mgu_predicate hd goal with
                                        
                                | None -> 
                                
(*             Printf.printf "matching %s with %s gives None\n%!"
                          (string_of_predicate goal)
                          (string_of_predicate hd); *)
                                
                                
                                ans_list (* This clause can't be matched *)
                                
                                
                                | Some subs -> 
                                
                                
(*             Printf.printf "matching %s with %s gives %s\n%!"
                          (string_of_predicate goal)
                          (string_of_predicate hd)
                          (string_of_substitution subs); *)
                          
                                    ans_list @
                                    
                                    (query_rec program
                                        (pred_list_subs subs (bdy @ other_goal_list))
                                        (pred_list_subs subs original_goal)
                                    )
            )
            
            []
            
            program

let query program goal = 
    
        query_rec program goal goal

val pred_list_subs : term Substitution.t -> predicate list -> predicate list =
  <fun>


val query_rec :
  clause list -> predicate list -> predicate list -> predicate list list =
  <fun>


val query : clause list -> predicate list -> predicate list list = <fun>


In [19]:
(* 5 points *)
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]] in
let g = [pred "f" [const_t "a"; const_t "b"]] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = g);
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]] in
let g = [pred "f" [const_t "a"; const_t "c"]] in
assert (query p g = [])

- : unit = ()


In [20]:
(* 5 points *)
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]] in
let g = [pred "f" [var_t "X"; const_t "b"]] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = [pred "f" [const_t "a"; const_t "b"]]);

let g = [pred "f" [var_t "X"; var_t "Y"]] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = [pred "f" [const_t "a"; const_t "b"]])

- : unit = ()


In [21]:
(* 5 points *)
let ancestor x y = pred "ancestor" [x;y] in
let father x y = pred "father" [x;y] in
let father_consts x y =  father (C (Constant x)) (C (Constant y)) in

let f1 = Fact (father_consts "rickard" "ned") in
let f2 = Fact (father_consts "ned" "robb") in
let r1 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Y")]) in
let r2 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Z"); ancestor (var_t "Z") (var_t "Y")]) in

let p = [f1;f2;r1;r2] in
let g = [ancestor (const_t "rickard") (const_t "ned")] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = g)

- : unit = ()


In [22]:
(* 5 points *)
(* Tests backtracking *)
let ancestor x y = pred "ancestor" [x;y] in
let father x y = pred "father" [x;y] in
let father_consts x y =  father (C (Constant x)) (C (Constant y)) in

let f1 = Fact (father_consts "rickard" "ned") in
let f2 = Fact (father_consts "ned" "robb") in
let r1 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Y")]) in
let r2 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Z"); ancestor (var_t "Z") (var_t "Y")]) in

let p = [f1;f2;r1;r2] in

let g = [ancestor (const_t "rickard") (const_t "robb")] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = g)

- : unit = ()


In [23]:
(* 10 points *)
(* Tests choice points *)
let ancestor x y = pred "ancestor" [x;y] in
let father x y = pred "father" [x;y] in
let father_consts x y =  father (C (Constant x)) (C (Constant y)) in

let f1 = Fact (father_consts "rickard" "ned") in
let f2 = Fact (father_consts "ned" "robb") in
let r1 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Y")]) in
let r2 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Z"); ancestor (var_t "Z") (var_t "Y")]) in

let p = [f1;f2;r1;r2] in

let g = [ancestor (var_t "X") (const_t "robb")] in
let g1,g2 = match query p g with [v1;v2] -> v1,v2 | _ -> failwith "error" in
assert (g1 = [ancestor (const_t "ned") (const_t "robb")]);
assert (g2 = [ancestor (const_t "rickard") (const_t "robb")])

- : unit = ()


In [24]:
(* 10 points *)
(* Tests choice points *)
let nil = const_t "nil" in
let cons h t = func_t "cons" [h;t] in
let append x y z = pred "append" [x;y;z] in
let c1 = fact @@ append nil (var_t "Q") (var_t "Q") in
let c2 = rule (append (cons (var_t "H") (var_t "P")) (var_t "Q") (cons (var_t "H") (var_t "R")))
              [append (var_t "P") (var_t "Q") (var_t "R")] in
let p = [c1;c2] in

let g = [append (var_t "X") (var_t "Y") (cons (const_t "1") (cons (const_t "2") (cons (const_t "3") nil)))] in

let g' = query p g in
assert (List.length g' = 4)

- : unit = ()
